In [19]:
import os
import sys
sys.path.append("D:\\Users\\NewUser\\Documents\\GitHub\\EEG-Classifiers-Ensemble\\")
sys.path.append("D:\\Users\\NewUser\\Documents\\GitHub\\EEG-Classifiers-Ensemble\\src")

from processing_eeg_methods.data_utils import get_dataset_basic_info
from processing_eeg_methods.share import datasets_basic_infos
import processing_eeg_methods.time_features.EEGExtract as eeg
from processing_eeg_methods.data_loaders import braincommand_dataset_loader_eeglab_set
import antropy as ant
import numpy as np
import pandas as pd
from scipy.stats import f_oneway

save_dir = r"D:\Users\NewUser\Downloads\Features"

dataset_name = "braincommand"

dataset_info =get_dataset_basic_info(datasets_basic_infos, dataset_name)


def get_entropy(data):
    entropy_values = []
    for data_trial in data:
        entropy_trial = []
        for data_channel in data_trial:
            # Convertir data_channel a un array unidimensional
            data_channel = np.array(data_channel).flatten()  
            entropy_trial.append(ant.higuchi_fd(data_channel))  # Aplicar higuchi_fd en el canal
        entropy_values.append(np.mean(entropy_trial))  # Promediar entropías por ensayo
    return entropy_values


def hjorth(X, D=None):
    if D is None:
        D = np.diff(X)
        D = D.tolist()

    D.insert(0, X[0])  # pad the first difference
    D = np.array(D)

    n = len(X)

    M2 = float(sum(D ** 2)) / n
    TP = sum(np.array(X) ** 2)
    M4 = 0
    for i in range(1, len(D)):
        M4 += (D[i] - D[i - 1]) ** 2
    M4 = M4 / n

    return np.sqrt(M2 / TP), np.sqrt(float(M4) * TP / M2 / M2)  # Hjorth Mobility and Complexity

def get_hjorth(data):
    values = []
    Complexity_values = []
    for data_trial in data:
        Mobility_trial = []
        Complexity_trial = []
        for data_channel in data_trial:
            Mobility, Complexity = hjorth(data_channel)
            Mobility_trial.append(Mobility)
            Complexity_trial.append(Complexity)
        values.append(np.mean(Mobility_trial))
        Complexity_values.append(np.mean(Complexity_trial))
    return values, Complexity_values

def get_lyapunov(data):
    lyapunov_values = []
    for data_trial in data:
        lyapunov_values.append(eeg.lyapunov(data_trial))
    return lyapunov_values

def extractions_train(data, labels):
    entropy = get_entropy(data)
    Mobility_values, Complexity_values = get_hjorth(data)
    lyapunov_values=get_lyapunov(data)
    lyapunov_values=np.array(lyapunov_values)
    df = pd.DataFrame({"entropy": entropy, "Complexity":Complexity_values, "Mobility":Mobility_values, "labels": labels})
    lyapunov_df = pd.DataFrame(lyapunov_values, columns=[f"Lyapunov_Channel_{i+1}" for i in range(lyapunov_values.shape[1])])  
    df = pd.concat([lyapunov_df, df], axis=1) 
    print("Characteristics extraction done")
    
    return df

def calculate_p_values(df, labels):
    p_values = {}
    for column in df.columns:
        if column != "labels": 
            groups = [df[column][labels == label] for label in np.unique(labels)]
            stat, p_value = f_oneway(*groups)
            p_values[column] = p_value

    p_values_df = pd.DataFrame(list(p_values.items()), columns=["Feature", "p-value"])
    return p_values_df

In [20]:

excel_file = f"{save_dir}\\all_subjects_data.xlsx"

with pd.ExcelWriter(excel_file, engine='openpyxl', mode='w') as writer:  # Cambiar 'w' a 'a' si ya existe el archivo
    for subject_id in [1, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 23, 25, 26, 27, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41]:
        game_mode = "calibration3"
        
        
        data, labels, ch_names = braincommand_dataset_loader_eeglab_set("", subject_id, game_mode)
        dataset_info["channels_names"] = ch_names
        
        
        df = extractions_train(data, labels)

        p_values = calculate_p_values(df, labels)

        df = pd.concat([df, p_values], axis=1)
        
        
        sheet_name = f"subject_{subject_id}_{game_mode}"
        
        
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"Datos guardados en la hoja: {sheet_name}")
    
    for subject_id in [1, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 23, 25, 26, 27, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41]:
        game_mode = "singleplayer"
        
        
        data, labels, ch_names = braincommand_dataset_loader_eeglab_set("", subject_id, game_mode)
        dataset_info["channels_names"] = ch_names
        
        
        df = extractions_train(data, labels)
        
        p_values = calculate_p_values(df, labels)

        df = pd.concat([df, p_values], axis=1)
        
        
        sheet_name = f"subject_{subject_id}_{game_mode}"
        
        
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"Datos guardados en la hoja: {sheet_name}")


Extracting parameters from D:\Users\NewUser\Downloads\cleaned\1_calibration3_cleaned.set...
Not setting metadata
171 matching events found
No baseline correction applied
0 projection items activated
Ready.
Characteristics extraction done
Datos guardados en la hoja: subject_1_calibration3
Extracting parameters from D:\Users\NewUser\Downloads\cleaned\8_calibration3_cleaned.set...
Not setting metadata
166 matching events found
No baseline correction applied
0 projection items activated
Ready.
Characteristics extraction done
Datos guardados en la hoja: subject_8_calibration3
Extracting parameters from D:\Users\NewUser\Downloads\cleaned\10_calibration3_cleaned.set...
Not setting metadata
205 matching events found
No baseline correction applied
0 projection items activated
Ready.
Characteristics extraction done
Datos guardados en la hoja: subject_10_calibration3
Extracting parameters from D:\Users\NewUser\Downloads\cleaned\11_calibration3_cleaned.set...
Not setting metadata
137 matching even

In [14]:
subject_id = 1
game_mode = "calibration3"
data, labels, ch_names = braincommand_dataset_loader_eeglab_set(
    "", subject_id, game_mode
)
dataset_info["channels_names"] = ch_names

df = extractions_train(data, labels)

file_name = f"{save_dir}\\subject_{subject_id}_{game_mode}.xlsx"
    
df.to_excel(file_name, index=False)
    
print(f"Archivo guardado: {file_name}")

Extracting parameters from D:\Users\NewUser\Downloads\cleaned\1_calibration3_cleaned.set...
Not setting metadata
171 matching events found
No baseline correction applied
0 projection items activated
Ready.
Characteristics extraction done
Archivo guardado: D:\Users\NewUser\Downloads\Features\subject_1_calibration3.xlsx
